In [77]:
from pathlib import Path

import numpy as np
import pandas as pd

seeds = np.array([
    432986705,
    28073546,
    1364097901,
    88338513,
    2733524843,
    234912494,
    3151642679,
    224376393,
    485709676,
    344068331,
    1560394266,
    911616092,
    3819746175,
    87998136,
    892394515,
    435690182,
    4218056486,
    23868437,
    848725444,
    8940450,
])

titles = [
    "seed-to-soil map:",
    "soil-to-fertilizer map:",
    "fertilizer-to-water map:",
    "water-to-light map:",
    "light-to-temperature map:",
    "temperature-to-humidity map:",
    "humidity-to-location map:",
]

mappings = {
    "seed": "soil",
    "soil": "fertilizer",
    "fertilizer": "water",
    "water": "light",
    "light": "temperature",
    "temperature": "humidity",
    "humidity": "location",
}

inv_mappings = {v: k for k, v in mappings.items()}

used_titles = titles.copy()
maps = {}
with open("input.txt") as f:
    ranges = []
    # while len(used_titles) > 0:
    maps = {}
    current_title = None
    for line in f:
        #print(current_title)
        if not line.strip():
            maps[current_title] = ranges.copy()
            ranges = []
            current_title = None
            continue
        if line.strip() in titles:
            current_title = line.split('-')[0]
            continue
        if current_title:
            ranges.append([int(l) for l in line.split()])
    maps[current_title] = ranges.copy()
    ranges = []
    current_title = None



# Convert list of 3 ranges to a pandas dataframe
dfs = {}
source = seeds
df_map = pd.DataFrame()

df_map["seed"] = seeds

# for i,map_ranges in enumerate(maps):
#     source_name = map_ranges
#     dest_name = mappings[map_ranges]
#     source = df_map[source_name].values

#     df = pd.DataFrame(maps[map_ranges], columns=["dest_start", "source_start", "length"])
#     df["dest_end"] = df["dest_start"] + df["length"]
#     df["source_end"] = df["source_start"] + df["length"]
#     df["offset"] = df["dest_start"] - df["source_start"]

#     df["source_items"] = df.apply(lambda row: source[(source >= row["source_start"]) & (source <= row["source_end"])], axis=1)


#     df["dest_items"] = df[["source_items", "offset"]].apply(lambda row: row["source_items"] + row["offset"], axis=1)

#     test_val: int = source[0]
#     df["source_items"][test_val in df["source_items"]]

In [116]:
sdf

,start,length,end,type
0,432986705,28073546,461060251,seed
1,485709676,344068331,829778007,seed
2,848725444,8940450,857665894,seed
3,892394515,435690182,1328084697,seed
4,1364097901,88338513,1452436414,seed
5,1560394266,911616092,2472010358,seed
6,2733524843,234912494,2968437337,seed
7,3151642679,224376393,3376019072,seed
8,3819746175,87998136,3907744311,seed
9,4218056486,23868437,4241924923,seed


In [137]:
dfs = {}
seed_ranges = seeds.reshape(-1, 2)

sdf = pd.DataFrame(seed_ranges, columns=["start", "length"]).sort_values(by="start").reset_index(drop=True)
sdf["end"] = sdf["start"] + sdf["length"]
sdf["type"] = "seed"

def interval_intersect(a_min, a_max, b_min, b_max):
    return (a_min <= b_max) and (b_min <= a_max)


# soil = pd.DataFrame(maps["seed"], columns=["dest_start", "start", "length"]).sort_values(by="start").reset_index(drop=True)
# soil["end"] = soil["start"] + soil["length"]
# soil["dest_end"] = soil["dest_start"] + soil["length"]

# soil[["start", "end", "dest_start", "dest_end"]]

#print(sdf)
#print(soil)

def get_intersections(start, end, map_name: str):

    next_map = mappings.get(map_name)
    if next_map == None:
        return map_name, start

    #for i, map_name in enumerate(maps):
    df = pd.DataFrame(maps[map_name], columns=["dest_start", "start", "length"]).sort_values(by="start").reset_index(drop=True)
    df["end"] = df["start"] + df["length"]
    df["dest_end"] = df["dest_start"] + df["length"]

    #for s, e in zip(curr_start, curr_end):
    idf = df[df[["start", "end"]].apply(lambda row: interval_intersect(start, end, row.start, row.end), axis=1)]

    if map_name == "humidity":
        return min(idf.dest_start)

    #starts = []
    for start, end in idf[["dest_start", "dest_end"]].values:
        return get_intersections(start, end, next_map)
    
    

    #return starts
        

    #return idf[["dest_start", "dest_end"]]
    
        

# get_intersections()

ints = []
for start, end in sdf[["start", "end"]].values:
    starts = get_intersections(start,end, "seed")
    #np.array(starts, dtype=)
    print(start, end, starts)

    if starts:
        ints.append(starts)
print(min(ints))
    

    # curr_start = start
    # curr_end = end
    # map_name = "seed"
    # next_map = mappings.get(map_name)

    # if next 

    # for i, map_name in enumerate(maps):
    #     #next_map = mappings.get(map)
    #     df = pd.DataFrame(maps[map_name], columns=["dest_start", "start", "length"]).sort_values(by="start").reset_index(drop=True)
    #     df["end"] = df["start"] + df["length"]
    #     df["dest_end"] = df["dest_start"] + df["length"]

    #     idf = df[df[["start", "end"]].apply(lambda row: interval_intersect(curr_start, curr_end, row.start, row.end), axis=1)]

        






# for i, map_name in enumerate(maps):
#     intersections = []
#     df = pd.DataFrame(maps[map_name], columns=["dest_start", "start", "length"]).sort_values(by="start").reset_index(drop=True)
#     df["end"] = df["start"] + df["length"]
#     df["dest_end"] = df["dest_start"] + df["length"]

#     if i == 0:
#         df["type"] = sdf

#     for start, end in sdf[["start", "end"]].values:
#         idf = df[df[["start", "end"]].apply(lambda row: interval_intersect(start, end, row.start, row.end), axis=1)]

#         idf["seed"] = start
#         idf["seed_end"] = end


#         #print(idf[["dest_start", "dest_end", "start", "end"]])
#         #idf[]
#         #print(start,end)
        

#         intersections.append(idf)

#         #interval_intersect(start, end, soil.start, soil.end)
#     #break
#     dff = pd.concat(intersections).sort_values("dest_start")
#     print(dff[["dest_start", "dest_end", "start", "end", ]])
#     break

# # fert = soil[["dest_start", "dest_end"]].copy().sort_values("dest_start").reset_index(drop=True)
# # fert.rename(columns={"dest_start": "start", "dest_end": "end"}, inplace=True)

# # soil = soil[["start", "end"]]

# fert

432986705 461060251 1466122599
485709676 829778007 1466122599
848725444 857665894 1219800446
892394515 1328084697 1219800446
1364097901 1452436414 1466122599
1560394266 2472010358 1466122599
2733524843 2968437337 897459980
3151642679 3376019072 None
3819746175 3907744311 1754676114
4218056486 4241924923 1411174643
897459980


In [93]:

dfs = {}
seed_ranges = seeds.reshape(-1, 2)

sdf = pd.DataFrame(seed_ranges, columns=["start", "length"])
sdf["end"] = sdf["start"] + sdf["length"]
sdf["type"] = "seed"

def interval_intersect(a_min, a_max, b_min, b_max):
    return (a_min <= b_max) and (b_min <= a_max)


for map_name in maps:
    df = pd.DataFrame(maps[map_name], columns=["dest_start", "start", "length"]).sort_values(by="start").reset_index(drop=True)
    df["end"] = df["start"] + df["length"]

    df["type"] = mappings[map_name]

    dff = pd.concat([sdf[["start", "end", "type"]], df[["start", "end", "type"]]]).sort_values(by="start").reset_index(drop=True)
    dfi = dff[(dff.end-dff.start.shift(-1))>0]
    
    sdf = dfi[dfi["type"]==mappings[map_name]].copy()

    

    print(map_name)
    print(dfi)

    #sdf[["source_start", "end"]].apply(lambda row: 
    #print(df.head())

print(dfi)

seed
         start         end  type
5    334379348   611890060  soil
7    485709676   829778007  seed
11   775565564   993160626  soil
13   892394515  1328084697  seed
14   993160626  1581788887  soil
16  1560394266  2472010358  seed
28  2650943102  2917079842  soil
29  2733524843  2968437337  seed
33  3039830108  3159586797  soil
34  3151642679  3376019072  seed
41  3770860054  4081180892  soil
44  4132946445  4294967296  soil
soil
         start         end        type
1    269225056   443504340  fertilizer
2    334379348   611890060        soil
5    744431503   909398162  fertilizer
6    775565564   993160626        soil
7    909398162  1172578553  fertilizer
8    993160626  1581788887        soil
17  2648616968  2883283037  fertilizer
18  2650943102  2917079842        soil
20  3015971185  3265637025  fertilizer
27  3714909503  4138946628  fertilizer
29  4132946445  4294967296        soil
fertilizer
         start         end        type
2    177937131   286114334       water
3   

In [92]:
min_range = None
for map_name in list(maps.keys())[::-1]:
    if map_name!= "seed":
        #print(map_name, inv_mappings[map_name])

        df = pd.DataFrame(maps[map_name], columns=["dest_start", "source_start", "length"])

        if not min_range:
            df.sort_values(by="dest_start", inplace=True)
            df.reset_index(drop=True, inplace=True)
            dfmin = df[df.dest_start==370325921]
        else:
            dfmin = df[(df.dest_start >= min_range[0]) & (df.dest_start <= min_range[1])]
        print(dfmin)

        min_range = (dfmin["source_start"].iloc[0], dfmin["source_start"].iloc[0] + dfmin["length"].iloc[0])
    
seed_ranges = seeds.reshape(-1, 2)

print(min_range)

def interval_intersect(a_min, a_max, b_min, b_max):
    return (a_min <= b_max) and (b_min <= a_max)

if min_range is None:
    raise ValueError("No valid range found")
intervals = [min_range]
for start, length in seed_ranges:


    end = start + length

    #print(start, length, end)
    assert (end > start)
    #assert end < start
    # print(start, end)
    intervals.append((start, end))


# print(min_range)
# print(intervals)

intervals = np.array(intervals)
dfi = pd.DataFrame(intervals,columns=['start','end'])
dfi.sort_values(by='start', inplace=True)
print(dfi[(dfi.end-dfi.start.shift(-1))>0])
dfi
    # if start >= min_start and start <= min_end and end >= min_start and end <= min_end:
    #     print(start, length, min_range)

Empty DataFrame
Columns: [dest_start, source_start, length]
Index: []


IndexError: single positional indexer is out-of-bounds

In [12]:
source_name = "seed"
dest_name = "soil"
source = df_map[source_name].values

df = pd.DataFrame(maps[source_name], columns=["dest_start", "source_start", "length"])
df["dest_end"] = df["dest_start"] + df["length"]
df["source_end"] = df["source_start"] + df["length"]
df["offset"] = df["dest_start"] - df["source_start"]

df["source_items"] = df.apply(lambda row: source[(source >= row["source_start"]) & (source <= row["source_end"])], axis=1)


df["dest_items"] = df[["source_items", "offset"]].apply(lambda row: row["source_items"] + row["offset"], axis=1)



df_map[dest_name] = df_map[source_name]

for i, s in enumerate(df_map[source_name]):
    row_s = np.where(df["source_items"].apply(lambda x: s in x))[0]
    if not row_s:
        continue

    ind = row_s[0]
    df_map.loc[i, dest_name] = df_map.loc[i, dest_name] + df.loc[ind, "offset"]

df_map[df_map["seed"] == df_map["soil"]]
                          
#df["source_items"].apply()

,seed,soil


In [16]:
seed_ranges

array([[ 432986705,   28073546],
       [1364097901,   88338513],
       [2733524843,  234912494],
       [3151642679,  224376393],
       [ 485709676,  344068331],
       [1560394266,  911616092],
       [3819746175,   87998136],
       [ 892394515,  435690182],
       [4218056486,   23868437],
       [ 848725444,    8940450]])

In [138]:
import math
import os, sys
import time
from typing import List

class SeedMap:
    def __init__(self, off: str, to: str):
        self.off = off
        self.to = to
        # tuple of (source, max_source, target)
        # source is inclusive, max_source is exclusive
        self.map_values: list[tuple[int, int, int]] = []
    # add a mapping to the list
    # ranges are non-overlapping
    def add(self, target: int, source: int, offset: int):
        self.map_values.append((source, source + offset, target))
    def sort(self):
        self.map_values.sort(key=lambda x: x[0])
    def get_destination(self, ranges: "SeedRangeList"):
        new_ranges = SeedRangeList()
        # for each range of seeds
        for start, end in ranges:
            # if we fully mapped the range, we can continue with next range
            found_mapping = False
            # for each mapping range
            for source, max_source, target in self.map_values:
                # if start is before range
                if start < source:
                    # if end is before range
                    if end < source:
                        # we can add this range
                        # ---- start -- end -- source -- max_source ----
                        new_ranges.add(start, end)
                        found_mapping = True
                        break
                    # else if end is in between range
                    if end <= max_source:
                        # we add only until source
                        # ---- start -- source -- end -- max_source ----
                        new_ranges.add(start, source)
                        # and map the rest
                        # ---- target -- target + (end - source) -- target + (max_source - source) ----
                        new_ranges.add(target, target + (end - source))
                        found_mapping = True
                        break
                    # else if end is after range
                    # we add only until source
                    # ---- start -- source -- max_source -- end ----
                    new_ranges.add(start, source)
                    # and map in between
                    # ---- target -- target + (max_source - source) ----
                    new_ranges.add(target, target + (max_source - source))
                    # we continue with the rest of the range
                    start = max_source
                # else if start is in between map range
                elif source <= start < max_source:
                    # if end is in between range
                    if end <= max_source:
                        # we map all
                        # ---- source -- start -- end -- max_source ----
                        # ---- target -- target + (start - source) -- target + (end - source) -- target + (max_source - source) ----
                        new_ranges.add(target + (start - source), target + (end - source))
                        found_mapping = True
                        break
                    # else if end is after this range
                    # we map until max_source
                    # ---- source -- start -- max_source -- end ----
                    # ---- target -- target + (start - source) -- target + (max_source - source) -- target + (end - source) ----
                    new_ranges.add(target + (start - source) , target + (max_source - source))
                    # we continue with the rest of the range
                    start = max_source
                # else if start is after range
                # we continue with the rest of the range
            # if we didn't find a mapping, we add the remaining range
            if not found_mapping:
                new_ranges.add(start, end)
        return new_ranges
    def __str__(self) -> str:
        return f"{self.off} => {self.to}"
    def __repr__(self) -> str:
        return self.__str__()

class SeedRangeList(List):
    def __init__(self):
        # start inclusive, end exclusive
        self.ranges: list[tuple[int, int]] = []
    def add(self, start: int, end: int):
        # for each already added range
        for i, (s, e) in enumerate(self.ranges):
            # if start is before range
            if start < s:
                # if end is before range
                if end < s:
                    # ---- start -- end -- s -- e ----
                    # we can add this range
                    self.ranges.insert(i, (start, end))
                    return
                # else if end is in between range
                if end <= e:
                    # we add only until s
                    # ---- start -- s -- end -- e ----
                    self.ranges.insert(i, (start, s))
                    return
                # else if end is after range
                # we add only until s
                # ---- start -- s -- e -- end ----
                self.ranges.insert(i, (start, s))
                # we continue with the rest of the range
                start = e
            # else if start is in between range
            elif start <= s <= end:
                # if end is in between range
                if end <= e:
                    # we already have this range
                    # ---- s -- start -- end -- e ----
                   return
                # else if end is after this range
                # we continue with the rest of the range
                # ---- s -- start -- e -- end ----
                start = e
            # else if start is after range
            # we continue with the rest of the range
            # ---- s -- e -- start -- end ----
        # if we haven't returned yet, we add the remaining range
        self.ranges.append((start, end))
    def __iter__(self):
        return iter(self.ranges)
    def __str__(self) -> str:
        return str(self.ranges)
    def __repr__(self) -> str:
        return self.__str__()


def main():
    with open("input.txt", "r", encoding="utf-8") as f:
        text = f.read().strip()
    lines = text.split("\n")
    # split seeds
    # example input "seeds: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14"
    seed_range = lines[0].split(": ")[1].split(" ")
    # example output ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", ...]
    seeds = SeedRangeList()
    # for each pair of seed inputs
    for i in range(0, len(seed_range), 2):
        # add the range to the list
        seeds.add(int(seed_range[i]), int(seed_range[i]) + int(seed_range[i+1]))
    # create list of seed maps
    seed_maps: list[SeedMap] = []
    # start reading first map
    current_map = None
    # for each line after seeds
    for line in lines[1:]:
        # skip empty lines
        if line == "":
            continue
        # if new map declaration begins
        if line.endswith("map:"):
            # get from and to strings (just for printing)
            off, to = line.split(" ")[0].split("-to-", 1)
            # create new map
            current_map = SeedMap(off, to)
            # and add it to the list
            seed_maps.append(current_map)
        else:
            # if not new map declaration, add the mapping
            current_map.add(*[int(s) for s in line.split(" ")])
    # we sort seed map ranges, so they are ascending
    # seed map ranges are not overlapping
    # later on, we can early break out as later ranges will start further than the current seed
    for seed_map in seed_maps:
        seed_map.sort()
    
    # for each mapping
    for seed_map in seed_maps:
        print(seed_map)
        # map all seed ranges to new destination
        seeds = seed_map.get_destination(seeds)
    
    # print the minimum location, which will be the start of a range
    print(min([start for start, _ in seeds]))

main()       



seed => soil
soil => fertilizer
fertilizer => water
water => light
light => temperature
temperature => humidity
humidity => location
15880236
